# Advanced RAG

## Context

Due to the substantial increase in the context window size and the decrease in token costs. RAG may not be required for many tasks (or at least not all of its components) since all the data can fit into the context window. 

However, if the context size is still insufficient, different methods of summarization and context compression have been devised.

In [ ]:
from langchain.memory import ConversationSummaryMemory, ConversationChain
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
llm=llm,
memory=ConversationSummaryMemory(llm=OpenAI()),
verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")

## Knowledge Graphs

As data volume increases, navigating and utilizing it effectively becomes crucial. Traditional RAG systems often fail to interpret context from data structure or metadata. For example, an LLM might overlook a company's phone number on a wiki page because its relevance isn't explicitly stated. Humans infer meaning from the data’s context, such as its location in a "Company Information" subdirectory.  

This gap can be addressed using **Knowledge Graphs with Metadata** (or **Knowledge Maps**). These structures provide not only raw data but also its relationships and storage context, enabling better understanding and retrieval by LLMs. This approach is termed **Graph Retrieval-Augmented Generation (GraphRAG)**.  

Knowledge Graphs are particularly effective for representing complex, interconnected information, capturing relationships and attributes that traditional vector databases struggle to manage.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*vtfNU6LNfY3rEClJ' width=300>

### Creating a **Knowledge Graph**

Creating a **Knowledge Graph** involves collecting, structuring, and connecting data to represent entities and their relationships. While traditionally a manual process requiring expertise in the subject and graph modeling, it can now be significantly automated using **LLMs**. Here's an overview of the steps:

1. **Data Collection**  
   - Gather data from various sources (e.g., documents, databases, websites).  
   - Ensure data is relevant and comprehensive for the target domain.

2. **Entity Recognition and Extraction**  
   - Identify key entities (e.g., people, organizations, locations) in the data.  
   - LLMs can analyze textual data to extract entities with context.  

3. **Relationship Identification**  
   - Determine how entities relate to one another (e.g., "works at," "is located in").  
   - LLMs can infer relationships by analyzing sentence structure, context, and semantics.  

4. **Graph Modeling**  
   - Decide on a graph schema, defining nodes (entities), edges (relationships), and attributes (properties).  
   - Structure the graph to reflect the data’s domain-specific intricacies.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*w5CqH0o-eJSlbPTS' width=500>

5. **Automating Graph Creation with LLMs**  
   - Use LLMs to:  
     - Suggest entities and relationships.  
     - Recommend schema design.  
     - Generate metadata and contextual connections.  

6. **Validation and Refinement**  
   - Verify the accuracy of extracted entities and relationships.  
   - Correct errors or refine connections where needed.

7. **Implementation in Graph Database**  
   - Store the graph in a database (e.g., Neo4j, ArangoDB).  
   - Leverage graph database tools for efficient querying and analysis.

8. **Visualization and Use**  
   - Use graph visualization tools (e.g., Gephi, Cytoscape) to represent the graph visually.  
   - Integrate the graph into applications for insights, search, or retrieval tasks.

**Advantages of LLMs in Knowledge Graph Creation**  
- **Automation:** Speeds up the traditionally manual process of entity and relationship extraction.  
- **Context Understanding:** Enhances accuracy by leveraging semantic and contextual knowledge.  
- **Scalability:** Handles large datasets efficiently, making it feasible to create graphs from extensive data sources.

## Access Control

Access to data is often uneven due to roles and permissions, posing challenges in search systems like vector databases. This highlights the need for effective **Access Management**, which is governed by models such as:  

1. **Role-Based Access Control (RBAC):** Access based on user roles.  
2. **Attribute-Based Access Control (ABAC):** Access based on attributes of users or resources.  
3. **Relationship-Based Access Control (ReBAC):** Access based on relationships between entities.  

These models can be combined and operate like graphs, where access is determined by checking if a path exists between the user and the resource.  

To ensure this functionality:  
- **Metadata Preservation:** Permissions and categories must be stored as metadata during **Data Ingestion** into knowledge graphs and vector databases.  
- **Search Filtering:** Searches must verify access rights for retrieved documents to match user permissions.  

Some advanced vector databases already support access control. However, if data is embedded into an LLM during training, access control is limited, relying on the LLM’s internal logic—an unreliable approach.  

**Alternative Solutions:**  
- **Censorship Layer:** Use a filtering mechanism (like Lakera or similar products) to block unauthorized outputs from the LLM.

## Ingestion and Parsing

Inserting data into a **graph** and **vector database** poses challenges, especially with formats like **PDF**, which can contain diverse data types (tables, images, text, graphics). Extracting structured data from PDFs, particularly nested tables, is often difficult and inefficient, frustrating many data scientists.

**Key Challenges with PDF Parsing:**
- **Complexity of Format:** PDFs can mix text, tables, and images in ways that make parsing difficult.
- **Lack of Reliable Solutions:** While libraries like **LLama Parse** attempt to extract data, they have varying degrees of success.
- **OCR as a Workaround:** In some cases, **Optical Character Recognition (OCR)** is used to extract data from document images, but this is not ideal for structured data extraction.

**Alternative Approaches:**
- There is no **Specialized PDF Parsing Models** yet. The focus is now shifting toward improving **answer quality** in systems. This can involve integrating knowledge graphs and exploring other methods to better handle the data structure and retrieval processes.

### CRAG (Corrective RAG)

In RAG, incorrect results can sometimes occur, and several methods can be used to evaluate and improve the accuracy of the generated responses:

1. **LLM-based Evaluation:** The LLM itself (or a lighter version) can assess the relevance of the results.
2. **Prompt Correction:** Adjusting the prompt to improve the retrieval or generation process.
3. **Graph Search:** Leveraging knowledge graphs to refine the search process.
4. **External Search Engines:** Using tools like Google Search to supplement the retrieval process and correct inaccuracies.

**CGRAG (Causal Graph Retrieval-Augmented Generation)** offers an extended framework that automates this evaluation and correction process. It uses a **state machine** approach that is built into another graph structure. This mechanism helps in automatically refining the results, providing a more efficient and structured way to handle the correction process.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*5Xn2f-SSi-Gs-5v2' width=500>

### Self-RAG

**Self-reflective RAG** is an advanced method based on the idea that incorporating self-reflection tokens into the process can improve the accuracy of results compared to traditional RAG. It builds upon concepts like **CRAG**, but adds an extra layer of self-reflection by fine-tuning the LLM to generate tokens that provide insight into its own reasoning. This method eliminates the need for guessing how confident the model is in its responses or how to handle uncertainties.

**Self-reflection Tokens:**
1. **Retrieve Token:** Determines whether the system needs to retrieve additional data chunks for a given prompt.
   - Options: **Yes**, **No**, **Continue**.

2. **ISREL Token:** Evaluates whether a specific chunk is relevant to the given prompt.
   - Options: **Relevant**, **Irrelevant**.

3. **ISSUP Token:** Assesses whether the LLM's response to a chunk is supported by the chunk itself.
   - Options: **Fully supported**, **Partially supported**, **No support**.

4. **ISUSE Token:** Rates whether the response generated by the LLM to a chunk is useful for answering the query.
   - Scale: **5 (most useful)** to **1 (least useful)**.

**State Machine and LangGraph:**
By using these self-reflection tokens, a state machine can be built within a **LangGraph** framework, enabling dynamic decision-making about the relevance, support, and usefulness of each chunk of data retrieved and the LLM's responses. This system automates evaluation and refinement, potentially leading to better, more accurate results in a RAG pipeline.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*pEkYymklCJGSwTnl' width=500>

### HyDe

**HyDe (Hypothetical Document Embeddings)** is a method that modifies the typical RAG process. Instead of using the user's question to search the vector database, HyDe generates a **hypothetical document** (a response) using the LLM. This generated response is then used to search for similar answers in the vector database. The approach, based on the study "Precise Zero-Shot Dense Retrieval without Relevance Labels," aims to improve the precision of zero-shot dense retrieval by leveraging the LLM’s ability to generate relevant responses before performing the search.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*P995ITJt65ECK_iv' width=500>

# Reference

[RAG Architecture: Advanced RAG](https://pub.towardsai.net/rag-architecture-advanced-rag-3fea83e0d189)